<font size="+3"><strong>3. Clustering with Multiple Features</strong></font>

In the previous notebook, we built a K-Means model to create clusters of respondents to the Survey of Consumer Finances. We made our clusters by looking at two features only, but there are hundreds of features in the dataset that we didn't take into account and that could contain valuable information. In this notebook, we'll examine all the features, selecting five to create clusters with. After we build our model and choose an appropriate number of clusters, we'll visualize multi-dimensional clusters in a 2D scatter plot using something called principal component analysis (PCA).

One of the persistent issues we've had with this dataset is that it includes some outliers in the form of ultra-wealthy households. This didn't make much of a difference for our last analysis, so we're going to focus on families with net worth under $2 million.

In [ ]:
# Libraries
import pandas as pd
import plotly.express as px

from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# **1. Prepare Data**

## **1.1. Import**

In [ ]:
def wrangle(filepath):
    """
    Read SCF data file into ``DataFrame``.

    Returns only credit fearful households whose net worth is less than $2 million.

    Parameters
    ----------
    filepath : str
        Location of CSV file.
    """
    #read file into DataFrame
    df = pd.read_csv(filepath)
    mask = (df["HBUS"] == 1) & (df["NETWORTH"] < 2e6)
    df = df[mask]
    return df

In [ ]:
df = wrangle("SCFP2019.csv")
print(df.shape)
df.head()

(2879, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
80,17,171,7802.265717,1,62,4,12,4,1,0,...,3,5,5,5,2,7,9,9,4,4
81,17,172,8247.536301,1,62,4,12,4,1,0,...,3,5,5,5,2,7,9,9,4,4
82,17,173,8169.562719,1,62,4,12,4,1,0,...,3,5,5,5,2,7,9,9,4,4
83,17,174,8087.704517,1,62,4,12,4,1,0,...,3,5,5,5,2,7,9,9,4,4
84,17,175,8276.510048,1,62,4,12,4,1,0,...,3,5,5,5,2,7,9,9,4,4


## **1.2. Explore**

In [ ]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)
top_ten_var

HOUSES      7.840298e+10
FIN         8.612408e+10
ACTBUS      8.681787e+10
BUS         8.996437e+10
KGBUS       9.247123e+10
KGTOTAL     1.178066e+11
NHNFIN      1.280493e+11
NFIN        2.450697e+11
NETWORTH    2.905111e+11
ASSET       3.904398e+11
dtype: float64

In [ ]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x = top_ten_var,
    y = top_ten_var.index,
    title = "SCF: High Variance Features"
)

fig.update_layout(xaxis_title = "Variance", yaxis_title = "Feature")

fig.show()

In [ ]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame = df,
    x = "NHNFIN",
    title = "Distribution of Non-home, Non-Financial Assets"
    )
fig.update_layout(xaxis_title = "Value [$]")

fig.show()

In [ ]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var, limits=(0.1,0.1)).sort_values().tail(10)
top_ten_trim_var

DEBT        1.339289e+10
ACTBUS      1.931986e+10
BUS         2.025053e+10
KGTOTAL     2.064910e+10
FIN         2.559217e+10
HOUSES      2.672881e+10
NHNFIN      4.427243e+10
NFIN        9.748428e+10
NETWORTH    1.507236e+11
ASSET       1.822011e+11
dtype: float64

In [ ]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x = top_ten_trim_var,
    y = top_ten_trim_var.index,
    title = "SCF: High Variance Features"
    )

fig.update_layout(xaxis_title ="Trimmed Variance", yaxis_title="Feature")

fig.show()

In [ ]:
# Generate a list high_var_cols with the column names of the five features with the highest trimmed variance

high_var_cols = top_ten_trim_var.tail(5).index.to_list()
high_var_cols

['HOUSES', 'NHNFIN', 'NFIN', 'NETWORTH', 'ASSET']

## **1.3. Split**

In [ ]:
# Create the feature matrix X

X = df[high_var_cols]
print("X shape:", X.shape)
X.head()

X shape: (2879, 5)


,HOUSES,NHNFIN,NFIN,NETWORTH,ASSET
80,500000.0,224000.0,724000.0,237600.0,810600.0
81,500000.0,223000.0,723000.0,236600.0,809600.0
82,500000.0,224000.0,724000.0,237600.0,810600.0
83,500000.0,222000.0,722000.0,234600.0,808600.0
84,500000.0,223000.0,723000.0,237600.0,809600.0


# **2. Build Model**

## **2.1. Iterate**

In [ ]:
# Create a DataFrame X_summary with the mean and standard deviation for all the features in X

X_summary = X.aggregate(["mean","std"]).astype(int)
X_summary

,HOUSES,NHNFIN,NFIN,NETWORTH,ASSET
mean,275323,294580,569904,617802,781722
std,280005,357839,495045,538990,624851


In [ ]:
# Create a StandardScaler transformer, use it to transform the data in X

# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

X_scaled shape: (2879, 5)


,HOUSES,NHNFIN,NFIN,NETWORTH,ASSET
0,0.802539,-0.197276,0.311329,-0.705519,0.046223
1,0.802539,-0.200071,0.309309,-0.707374,0.044622
2,0.802539,-0.197276,0.311329,-0.705519,0.046223
3,0.802539,-0.202866,0.307288,-0.711086,0.043021
4,0.802539,-0.200071,0.309309,-0.705519,0.044622


In [ ]:
# Create a DataFrame X_scaled_summary with the mean and standard deviation for all the features in X_scaled

X_scaled_summary = X_scaled.aggregate(["mean","std"]).astype(int)
X_scaled_summary

,HOUSES,NHNFIN,NFIN,NETWORTH,ASSET
mean,0,0,0,0,0
std,1,1,1,1,1


In [ ]:
# Use a for loop to build and train a K-Means model where n_clusters ranges from 2 to 12 (inclusive)

n_clusters = range(2,13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    #Build model
    model = make_pipeline(
        StandardScaler(),
        KMeans(n_clusters=k, random_state=42, n_init=10)
        )
    #Train model
    model.fit(X)

    #Calculate inertia
    inertia_errors.append(model.named_steps['kmeans'].inertia_)

    #Calculate silhouette score
    silhouette_scores.append(
        silhouette_score(X, model.named_steps['kmeans'].labels_)
        )


print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Inertia: [6569.8549994766, 4826.377288713905, 3989.574242512027]

Silhouette Scores: [0.5655472571445666, 0.46403026638003597, 0.42745959548241746]


In [ ]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=inertia_errors, title="K-Mean Model: Inertia vs Number of Clusters"
    )

fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Inertia")

fig.show()

You can see that the line starts to flatten out around 4 or 5 clusters.

In [ ]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=silhouette_scores, title="K-Means Model: Silhouette Score vs Number of Clusters"
    )
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Silhouette score")
fig.show()

This one's a little less straightforward, but we can see that the best silhouette scores occur when there are 3 or 4 clusters.

Putting the information from this plot together with our inertia plot, it seems like the best setting for `n_clusters` will be 4.

In [ ]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42, n_init=10)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, n_init=10, random_state=42))])

# **3. Communicate**

It's time to let everyone know how things turned out. Let's start by grabbing the labels.

In [ ]:
# Extract labels

labels = final_model.named_steps["kmeans"].labels_
print(labels[:5])

[3 3 3 3 3]


In [ ]:
# Create a DataFrame xgb that contains the mean values of the features in X for each of the clusters

xgb = X.groupby(labels).mean()
xgb

,HOUSES,NHNFIN,NFIN,NETWORTH,ASSET
0,1.235506e+05,1.025904e+05,2.261410e+05,2.191609e+05,3.097677e+05
1,3.327272e+05,1.016225e+06,1.348952e+06,1.360742e+06,1.604207e+06
2,1.023240e+06,5.573765e+05,1.580616e+06,1.577759e+06,2.140209e+06
3,3.874465e+05,2.686546e+05,6.561011e+05,8.239263e+05,1.013029e+06


In [ ]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title="Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")
fig.show()

In [ ]:
# Create a PCA transformer, use it to reduce the dimensionality of the data in X to 2, and then put the transformed data into a DataFrame named X_pca

# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns= ["PC1","PC2"])

print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca shape: (2879, 2)


,PC1,PC2
0,-84088.086960,88084.477121
1,-85987.452691,88699.746640
2,-84088.086960,88084.477121
3,-88407.089687,89188.007765
4,-85467.181427,88826.755034


In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x= "PC1",
    y="PC2",
    color=labels.astype(str),
    title= "PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")

fig.show()